In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [2]:
import sentencepiece as spm
sp_model = spm.SentencePieceProcessor()
sp_model.Load('prepare/sp10m.cased.ms-en.model')

True

In [3]:
import tensorflow as tf
import tensorflow_text
import struct

unknown = b'\xff\xff\xff\xff'

def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    for node in graph_def.node:
        
        if node.op == 'RefSwitch':
          node.op = 'Switch'
          for index in xrange(len(node.input)):
            if 'moving_' in node.input[index]:
              node.input[index] = node.input[index] + '/read'
        elif node.op == 'AssignSub':
          node.op = 'Sub'
          if 'use_locking' in node.attr: del node.attr['use_locking']
        elif node.op == 'AssignAdd':
          node.op = 'Add'
          if 'use_locking' in node.attr: del node.attr['use_locking']
        elif node.op == 'Assign':
          node.op = 'Identity'
          if 'use_locking' in node.attr: del node.attr['use_locking']
          if 'validate_shape' in node.attr: del node.attr['validate_shape']
          if len(node.input) == 2:
            node.input[0] = node.input[1]
            del node.input[1]
            
        if 'Reshape/shape' in node.name or 'Reshape_1/shape' in node.name:
            b = node.attr['value'].tensor.tensor_content
            arr_int = [int.from_bytes(b[i:i + 4], 'little') for i in range(0, len(b), 4)]
            if len(arr_int):
                arr_byte = [unknown] + [struct.pack('<i', i) for i in arr_int[1:]]
                arr_byte = b''.join(arr_byte)
                node.attr['value'].tensor.tensor_content = arr_byte
            
            if len(node.attr['value'].tensor.int_val):
                node.attr['value'].tensor.int_val[0] = -1
    
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [6]:
with open('prepare/test_X') as fopen:
    X = fopen.read().split('\n')
    
with open('prepare/test_Y') as fopen:
    Y = fopen.read().split('\n')

In [5]:
g = load_graph('small/frozen_model.pb')
x = g.get_tensor_by_name('import/inputs:0')
logits = g.get_tensor_by_name('import/SelectV2_3:0')
test_sess = tf.InteractiveSession(graph = g)

In [8]:
from tqdm import tqdm

batch_size = 10
results = []
for i in tqdm(range(0, len(X), batch_size)):
    batch_x = X[i: i + batch_size]
    batches = []
    for b in batch_x:
        batches.append(f'grafik pengetahuan: {b}')
    g = test_sess.run(logits, feed_dict = {x:batches})
    results.extend(g.tolist())

100%|██████████| 1000/1000 [18:55<00:00,  1.14s/it]


In [9]:
results_Y = [sp_model.DecodeIds(r) for r in results]
results_Y[0], Y[0]

('Southern Decoration material used Silver, Gold, Gold point in time 00 1997, Gold point in time 00 2007, Gold point in time 00 2007.',
 'Southern Decadence Silver ( color ) together with Teal, Teal together with White, Teal together with Silver ( color ), Teal together with Gold ( color ), Teal together with Crimson, Teal point in time 00 1997, Teal point in time 00 2007.')

In [10]:
from tensor2tensor.utils import bleu_hook

In [11]:
bleu_hook.compute_bleu(reference_corpus = Y, 
                       translation_corpus = results_Y)

0.91023386